In [ ]:
import nltk
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
import requests
import glob
import os
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
import string
import re
import openpyxl
from openpyxl import Workbook,load_workbook

In [ ]:
df = pd.read_excel('input.xlsx')
print(df)

In [ ]:
stop_words = []
final_stop_words = []

# Define the directory path where the text files are located
dir = 'C:\\Users\\vatsa\\Python\\StopWords'  # Replace with the actual directory path

# Use the glob module to retrieve a list of text files in the directory
file_paths = glob.glob(dir + '/*.txt')

for f in file_paths:
    with open(f, 'r') as file:
        content = file.read()
        words = content.split()
        stop_words.extend(words)

for x in stop_words:
    final_stop_words.append(x.lower())

final_stop_words = set(final_stop_words)

In [ ]:
def remove_unicode(scrapped_data):
    no_unicode = []
    for i in scrapped_data:
        strencode = i.encode("ascii", "ignore")
        strdecode = strencode.decode()
        no_unicode.append(strdecode)
        
    return no_unicode

In [ ]:
for ind in df.index:
    
    URL_ID = df['URL_ID'][ind]
    URL = df['URL'][ind]
    url = URL
    
    html = requests.get(url)
    
    soup = BeautifulSoup(html.content,'html.parser') 
    
   
    title = soup.find_all('h1')
    result_1 = soup.find('div',class_=['td-post-content', 'tagdiv-type'])
    
    if result_1 != None and len(title) >= 1:     
        
        para = result_1.find_all('p')
        heading = str(title[0].text)
        
        paragraph=""
       
        for p in para:
            paragraph = paragraph + str(p.text)
            
        
        scrapped_data = heading + paragraph
        scrapped_data = scrapped_data.split()
        scrapped_data = remove_unicode(scrapped_data)
        scrapped_data = ' '.join(scrapped_data)
        
        
        file_name = str(URL_ID)+'.txt'
        with open('C:\\Users\\vatsa\\Python\\Extracted_words\\'+file_name,'w') as file:
            words = scrapped_data.split()
            for word in words:
                file.write(word + ' ')
                    
                    
        file.close()  

In [ ]:
def tokenize_fun(extracted_str):
    text = extracted_str
    tkn = word_tokenize(text)
    return tkn

In [ ]:
def tokenize_fun_sent(extracted_str):
    text = extracted_str
    tkn = sent_tokenize(text)
    return tkn

In [ ]:
def filter_stopwords(word_tokens):
    filtered_data = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_data.append(w)
            
    return filtered_data

In [ ]:
positive_words = set()
with open('C:\\Users\\vatsa\\Python\\MasterDictionary\\positive-words.txt','r') as file:   
    for line in file:       
        for word in line.split():    
            positive_words.add(word)
            

In [ ]:
negative_words = set()
with open('C:\\Users\\vatsa\\Python\\MasterDictionary\\negative-words.txt','r') as file:   
    for line in file:       
        for word in line.split():    
            negative_words.add(word)
            

In [ ]:
def filter_symbols(filtered_data):
    english_words = []
    words = set(nltk.corpus.words.words())

    for i in filtered_data:
        i = " ".join(w for w in nltk.wordpunct_tokenize(i) if w.lower() in words or not w.isalpha())
        english_words.append(i);
        
    english_words_no_punctuations = []
    for i in english_words:
        i = i.translate(str.maketrans('', '', string.punctuation))
        english_words_no_punctuations.append(i)
    
    no_unicode = []
    for i in english_words_no_punctuations:
        strencode = i.encode("ascii", "ignore")
        strdecode = strencode.decode()
        no_unicode.append(strdecode)
    
    english_words_no_punctuations = no_unicode    
    final_filtered_data = set(english_words_no_punctuations)
    final_filtered_data = list(final_filtered_data)
    for i in final_filtered_data:
        if i=='':
            final_filtered_data.remove(i)
        
    return final_filtered_data

In [ ]:
def filter_inbuilt_stopwords_punctuations(word_tokens):
    inbuilt_stopwords = stopwords.words('english')
    
    data_no_stopwords = []
    for item in word_tokens:
        if item not in inbuilt_stopwords:
            data_no_stopwords.append(item)
            
    no_pun_stop = []
    for i in data_no_stopwords:
        i = i.translate(str.maketrans('', '', string.punctuation))
        no_pun_stop.append(i)
        
    return no_pun_stop

In [ ]:
def syllable_count(word):
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e") or word.endswith("es") or word.endswith("ed"):
        count -= 1
        
    if word.endswith("ted") or word.endswith("tes") or word.endswith("ses") or word.endswith("ied") or word.endswith("ies"):
        count += 1
                
    if count == 0:
        count += 1
    return count

In [ ]:
df_o = pd.DataFrame(columns=['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX_WORD_COUNT','WORD_COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])

directory = 'C:\\Users\\vatsa\\Python\\Extracted_words'

ff = os.listdir(directory)
sorted_files = sorted(ff)
for filename in sorted_files:
    extracted_list = []
    f = os.path.join(directory, filename)
    
    
    if os.path.isfile(f) and filename.endswith('.txt'):
        with open(f,'r') as file:
            for line in file:       
                extracted_list = line.split()
        file.close()
        
    
    extracted_str = ' '.join(extracted_list)

    word_tokens = tokenize_fun(extracted_str)
    sentence_tokens = tokenize_fun_sent(extracted_str)  
    filtered_data = filter_stopwords(word_tokens)
    final_filtered_data = filter_symbols(filtered_data)
    
        # 1.Positive score
    p_score = 0
    for item in final_filtered_data:
        if item in positive_words:
            p_score=p_score+1    
        
        # 2.Negative score
    n_score = 0;
    for item in final_filtered_data:
        if item in negative_words:
            n_score = n_score+1
          
        # 3.Polarity score
    pol_score = 0
    pol_score =(p_score-n_score)/((p_score + n_score)+0.000001)
    
        # 4.Subjectivity score
    sub_score = 0
    sub_score = (p_score+n_score)/len(extracted_list) + 0.000001
    
    # word length, sentence length 
    word_length = len(word_tokens)
    sentence_length = len(sentence_tokens)
    
    # 5.Average sentence length 8. average no. of words per sectence
    average_sentence_length = word_length/sentence_length
    average_no_of_words_per_sentence = average_sentence_length
    
    #9. Complex Words
    complex_words = 0
    for word in word_tokens:
        syllables = syllable_count(word)
        if syllables > 2:
            complex_words = complex_words + 1
            
            #total syllables in article
            total_syllables_in_article = 0
            total_syllables_in_article = total_syllables_in_article + syllables
            
    #10. Word Count (no inbuilt stopwords, no punctuations)
    no_pun_stop = filter_inbuilt_stopwords_punctuations(word_tokens)
    word_count = len(no_pun_stop)
    
    # 11.Average syllables per word
    syllables_per_word = total_syllables_in_article/word_length
    
    # 6.percentage of complex words
    complex_percent = complex_words/word_length
    
    # 7.fog index
    fog_index = 0.4*(average_sentence_length + complex_percent)
    
    # 12.Personal pronoun using regex
    count_pronouns = 0
    pronouns = r"\b(he|she|it|we|they|you|I)\b" #US not included
    matches = re.findall(pronouns, scrapped_data , re.IGNORECASE)
    count_pronouns = len(matches)
    
    # 13.Average Word length
    total_characters = 0
    for item in word_tokens:
        for character in item:
            total_characters = total_characters + 1
            
    average_word_length = total_characters/word_length
    
    url_id_ = filename[:-4]
    url = df.loc[df['URL_ID']==int(url_id_),'URL'].iloc[0]

    
    
    new_row = {'URL_ID' : url_id_ ,'URL' : url,'POSITIVE SCORE' : p_score,'NEGATIVE SCORE' : n_score,'POLARITY SCORE' : pol_score,'SUBJECTIVITY SCORE' : sub_score,'AVG SENTENCE LENGTH' : average_sentence_length,'PERCENTAGE OF COMPLEX WORDS' : complex_percent,'FOG INDEX' : fog_index,'AVG NUMBER OF WORDS PER SENTENCE' : average_no_of_words_per_sentence ,'COMPLEX_WORD_COUNT' : complex_words,'WORD_COUNT' : word_count,'SYLLABLE PER WORD' : syllables_per_word,'PERSONAL PRONOUNS' : count_pronouns,'AVG WORD LENGTH' : average_word_length}
    
    new_row_df = pd.DataFrame([new_row])
    df_o = pd.concat([df_o,new_row_df], ignore_index = True)
    
    
    
    
    

In [ ]:
print(df_o)

In [ ]:
df_o.to_excel('Output.xlsx', index=False)